In [1]:
import numpy as np
import pandas as pd

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from scipy import optimize
from sklearn.metrics import r2_score
from scipy.stats import t
from scipy import stats

<ipython-input-1-3ca75f38248d>:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Create connection.
cnx = sqlite3.connect("data/Marketing_DS_task_2.db")

revenue_data = pd.read_sql_query("SELECT * FROM revenue_data", cnx)
spend_data = pd.read_sql_query("SELECT * FROM spend_data", cnx)

In [3]:
spend_data_country_group = (
    spend_data.groupby(
        [
            "platform",
            "country_group",
            "channel",
            "media_type",
            "install_month",
        ]
    )
    .sum("spend")
    .reset_index()
)

In [4]:
spend_revenue_df = pd.merge(
    spend_data_country_group,
    revenue_data,
    how="inner",
    on=["platform", "country_group", "channel", "media_type", "install_month"],
)
spend_revenue_df.sort_values(
    by=[
        "install_month",
        "platform",
        "country_group",
        "channel",
        "media_type",
    ],
    inplace=True,
)

In [5]:
spend_revenue_df.replace("", np.nan, inplace=True)

# Check if 'Revenue' is in column names
for col in spend_revenue_df.columns:
    if "Revenue" in col:
        # Convert 'Revenue' column to float

        spend_revenue_df[col] = spend_revenue_df[col].astype(float)

spend_revenue_df["spend"] = spend_revenue_df["spend"].astype(float)
spend_revenue_df["installs"] = spend_revenue_df["installs"].astype(int)
spend_revenue_df["cpi"] = spend_revenue_df["spend"] / spend_revenue_df["installs"]
spend_revenue_df["install_month"] = pd.to_datetime(
    spend_revenue_df["install_month"], format="%Y-%m-%d"
)

<ipython-input-5-8ff3d5e5e963>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spend_revenue_df.replace("", np.nan, inplace=True)


In [6]:
data_predict = spend_revenue_df[
    (spend_revenue_df["installs"] > 0)
    & (spend_revenue_df["spend"] > 0)
    & (spend_revenue_df["D180_Revenue"].isna())
    # & (spend_revenue_df["D90_Revenue"].notna())
]

In [7]:
# mask na values or 0 spend values
spend_revenue_df = spend_revenue_df[
    (spend_revenue_df["installs"] > 0)
    & (spend_revenue_df["spend"] > 0)
    # & (spend_revenue_df["D30_Revenue"].notna())
    # & (spend_revenue_df["D120_Revenue"].notna())
]
spend_revenue_df.dropna(inplace=True)

In [8]:
# Check if 'Revenue' is in column names
for col in spend_revenue_df.columns:
    if "Revenue" in col:
        # Convert 'Revenue' column to float
        period = col.split("_")[0]
        spend_revenue_df[f"{period}_arpi"] = (
            spend_revenue_df[col] / spend_revenue_df["installs"]
        )
        spend_revenue_df.drop(columns=[col], inplace=True)


# Check if 'Revenue' is in column names
for col in data_predict.columns:
    if "Revenue" in col:
        # Convert 'Revenue' column to float
        period = col.split("_")[0]
        data_predict[f"{period}_arpi"] = data_predict[col] / data_predict["installs"]
        data_predict.drop(columns=[col], inplace=True)

spend_revenue_df.head(60)

,platform,country_group,channel,media_type,install_month,spend,installs,cpi,D1_arpi,D7_arpi,D14_arpi,D21_arpi,D30_arpi,D60_arpi,D90_arpi,D120_arpi,D150_arpi,D180_arpi
0,android,Group_1,Networks,Demand-Side-Platform,2022-01-01,71400.67,12706,5.619445,0.307365,0.789110,1.175714,1.479605,1.871305,2.933325,3.550394,3.974672,4.385082,3.974672
22,android,Group_1,Networks,Incentivized,2022-01-01,26427.15,21404,1.234683,0.235817,0.608880,0.869778,1.027187,1.145385,1.523574,1.809721,1.916510,1.981350,1.916510
44,android,Group_1,Networks,Pre-installed,2022-01-01,7563.02,4866,1.554258,0.094447,0.439166,0.634457,0.891270,1.150927,2.093726,2.863681,3.130423,3.252392,3.130423
67,android,Group_1,Networks,Video Networks,2022-01-01,6284.33,2582,2.433900,0.037184,0.111038,0.132405,0.135302,0.137405,0.142665,0.144241,0.150290,0.151340,0.150290
89,android,Group_1,Search,Search,2022-01-01,121033.79,19564,6.186556,0.065242,0.149986,0.226388,0.294749,0.373888,0.563881,0.695946,0.804624,0.879140,0.934009
111,android,Group_1,Social,Social,2022-01-01,13526.48,2978,4.542136,0.081437,0.167246,0.212760,0.255702,0.293183,0.375484,0.395406,0.461293,0.547085,0.547085
133,android,Group_2,Networks,Demand-Side-Platform,2022-01-01,19311.52,5950,3.245634,0.196718,0.371607,0.488635,0.565052,0.620509,0.895094,1.063188,1.163518,1.249499,1.163518
155,android,Group_2,Networks,Incentivized,2022-01-01,23608.26,8060,2.929065,0.396866,0.737143,0.886333,0.977524,1.070172,1.226228,1.278187,1.320700,1.376752,1.320700
177,android,Group_2,Networks,Pre-installed,2022-01-01,1048.26,240,4.367750,0.000000,0.000000,0.005625,0.048125,0.048125,0.050917,0.110458,0.130250,0.368708,0.130250
196,android,Group_2,Networks,Video Networks,2022-01-01,6047.21,2115,2.859201,0.120383,0.218804,0.337593,0.394884,0.515064,0.841035,1.198704,1.465390,1.740983,1.465390


In [9]:
data = spend_revenue_df.copy()

data.set_index(
    [
        "install_month",
        "platform",
        "country_group",
        "channel",
        # "media_type",
    ],
    inplace=True,
)

data.drop(columns=["spend", "installs", "cpi", "media_type"], inplace=True)

data = data.stack().reset_index().rename(columns={"level_4": "age", 0: "arpi"})
data["age"] = data["age"].str.replace("_arpi", "")
data["age"] = data["age"].str.replace("D", "").astype(int)
data["install_month"] = pd.to_datetime(data["install_month"], format="%Y-%m-%d")

In [10]:
def ltv_func(t, a, b):
    return a * np.log(t) + b

In [11]:
# Get unique values from each column
platform = data["platform"].unique()
country_group = data["country_group"].unique()
channel = data["channel"].unique()
install_month = data["install_month"].unique()


combinations = list(itertools.product(platform, country_group, channel, install_month))

In [12]:
r2_score_dict = {}
combination_weights = {}
ci_dict = {}

for combination in combinations:
    data_comb = data[
        (data["platform"] == combination[0])
        & (data["country_group"] == combination[1])
        & (data["channel"] == combination[2])
        & (data["install_month"] == combination[3])
    ]

    if (not data_comb.empty) & (data_comb.age.nunique() > 2):

        # Get the unique months and sort them
        unique_age = sorted(data_comb["age"].unique())

        # Find the month to split on - for example, the 80th percentile
        split_age = unique_age[int(len(unique_age) * 0.7)]

        # Create the train and test datasets
        train_data = data_comb[data_comb["age"] < split_age]
        test_data = data_comb[data_comb["age"] >= split_age]

        train_data = train_data.groupby(["age"])["arpi"].mean().reset_index()
        test_data = test_data.groupby(["age"])["arpi"].mean().reset_index()

        X_l = train_data["age"].values
        Y_l = train_data["arpi"].values

        coefs_l, cov = optimize.curve_fit(ltv_func, X_l, Y_l)
        combination_weights[combination] = coefs_l

        X = test_data["age"].values
        Y = test_data["arpi"].values
        # concat train and test X
        X = np.concatenate((X_l, X))
        Y = np.concatenate((Y_l, Y))
        fitted_func = ltv_func(X, coefs_l[0], coefs_l[1])

        residuals = Y - fitted_func
        sse = np.sum(residuals**2)
        stdev = np.sqrt(sse / (len(Y) - 2))

        confidence_level = 0.99
        degrees_freedom = len(Y) - 2
        t_value = t.ppf((1 + confidence_level) / 2, degrees_freedom)

        confidence_intervals = t_value * stdev / np.sqrt(len(Y))
        ci_dict[combination] = confidence_intervals

        # plt.plot(X, fitted_func, color="b")
        # plt.scatter(X, Y, color="r")
        # plt.fill_between(
        #     X,
        #     np.maximum(0, fitted_func - confidence_intervals),
        #     fitted_func + confidence_intervals,
        #     color="b",
        #     alpha=0.2,
        # )
        # plt.show()

        r2_score_dict[combination] = r2_score(Y, fitted_func)

np.mean(list(r2_score_dict.values()))
np.median(list(r2_score_dict.values()))

0.8530579135015952

In [13]:
r2_df = pd.DataFrame(
    [
        {
            "platform": key[0],
            "country_group": key[1],
            "channel": key[2],
            "install_month": key[3],
            "r2_score": value,
        }
        for key, value in r2_score_dict.items()
    ]
)


combination_df = pd.DataFrame(
    [
        {
            "platform": key[0],
            "country_group": key[1],
            "channel": key[2],
            "install_month": key[3],
            "weight1": value[0],
            "weight2": value[1],
        }
        for key, value in combination_weights.items()
    ]
)

ci_df = pd.DataFrame(
    [
        {
            "platform": key[0],
            "country_group": key[1],
            "channel": key[2],
            "install_month": key[3],
            "ci": value,
        }
        for key, value in ci_dict.items()
    ]
)

In [14]:
df_combined = pd.merge(
    r2_df,
    combination_df,
    on=["platform", "country_group", "channel", "install_month"],
)
df_combined = pd.merge(
    df_combined,
    ci_df,
    on=["platform", "country_group", "channel", "install_month"],
)
df_combined.head(60)

,platform,country_group,channel,install_month,r2_score,weight1,weight2,ci
0,android,Group_1,Networks,2022-01-01,0.841369,0.409262,-0.135418,0.379201
1,android,Group_1,Networks,2022-02-01,0.876068,0.573412,-0.112422,0.454837
2,android,Group_1,Networks,2022-03-01,0.882529,0.788059,-0.414703,0.566921
3,android,Group_1,Networks,2022-04-01,0.844192,0.465907,-0.137796,0.428862
4,android,Group_1,Networks,2022-05-01,0.691918,0.487555,-0.212065,0.762523
5,android,Group_1,Networks,2022-06-01,0.851696,0.491536,-0.058134,0.441823
6,android,Group_1,Networks,2022-07-01,0.837984,0.500768,-0.021762,0.476787
7,android,Group_1,Networks,2022-08-01,0.822632,0.512620,-0.205058,0.518790
8,android,Group_1,Networks,2022-09-01,0.806730,0.722466,-0.241964,0.779817
9,android,Group_1,Networks,2022-10-01,0.762663,0.629700,-0.219907,0.795656


In [15]:
# df_combined = df_combined[df_combined["r2_score"] > 0.75]

df_combined.drop(columns=["install_month"], inplace=True)

df_combined = (
    df_combined.groupby(["platform", "country_group", "channel"]).mean().reset_index()
)

In [16]:
data_p = data_predict.copy()

data_p.set_index(
    [
        "install_month",
        "platform",
        "country_group",
        "channel",
    ],
    inplace=True,
)

data_p.drop(columns=["spend", "installs", "cpi", "media_type"], inplace=True)

data_p = data_p.stack().reset_index().rename(columns={"level_4": "age", 0: "arpi"})
data_p["age"] = data_p["age"].str.replace("_arpi", "")
data_p["age"] = data_p["age"].str.replace("D", "").astype(int)
data_p["install_month"] = pd.to_datetime(data_p["install_month"], format="%Y-%m-%d")

In [17]:
# Get unique values from each column
platform = data_p["platform"].unique()
country_group = data_p["country_group"].unique()
channel = data_p["channel"].unique()
install_month = data_p["install_month"].unique()

combinations = list(itertools.product(platform, country_group, channel, install_month))

In [18]:
df_true_preds = pd.DataFrame()

for combination in combinations:
    data_comb = data_p[
        (data_p["platform"] == combination[0])
        & (data_p["country_group"] == combination[1])
        & (data_p["channel"] == combination[2])
        & (data_p["install_month"] == combination[3])
    ]
    if not data_comb.empty:
        test_data = data_comb.groupby(["age"])["arpi"].mean().reset_index()

        X_l = test_data["age"].values
        Y_l = test_data["arpi"].values

        X_complete = [1, 7, 14, 21, 30, 60, 90, 120, 150, 180]
        X_predict = [x for x in X_complete if x not in X_l]
        # X_predict.insert(0, X_l[-1])

        df_learned = df_combined[
            (df_combined["platform"] == combination[0])
            & (df_combined["country_group"] == combination[1])
            & (df_combined["channel"] == combination[2])
        ]

        weight_1 = df_learned["weight1"].values[0]
        weight_2 = df_learned["weight2"].values[0]
        ci = df_learned["ci"].values[0]

        Y = test_data["arpi"].values

        # Define the number of simulations
        n_simulations = 10000

        if max(X_l) >= 14:  # 120:
            coefs_l = np.nan
            coefs_l, cov = optimize.curve_fit(ltv_func, X_l, Y_l)

            # calculate ci
            stdev = np.sqrt(
                np.sum((Y - ltv_func(X_l, coefs_l[0], coefs_l[1])) ** 2) / (len(Y) - 2)
            )
            ci = t.ppf(0.99, len(Y) - 2) * stdev / np.sqrt(len(Y))

            weight_for_sim_0 = coefs_l[0]
            weight_for_sim_1 = coefs_l[1]
            if max(X_l) >= 120:
                eligable_prediction = True
            else:
                eligable_prediction = False
        else:
            weight_for_sim_0 = weight_1
            weight_for_sim_1 = weight_2
            eligable_prediction = False

        # Initialize an array to store the results
        ltv_simulation = np.zeros((n_simulations, len(X_predict)))

        # Run the Monte Carlo simulation
        for i in range(n_simulations):
            # Generate random weights for each simulation
            weight_1_sim = np.random.normal(weight_for_sim_0, ci)
            weight_2_sim = np.random.normal(weight_for_sim_1, ci)

            # Calculate the LTV for each time period
            ltv_simulation[i, :] = ltv_func(X_predict, weight_1_sim, weight_2_sim)

        # Calculate the mean and standard deviation of the LTV for each time period
        ltv_mean = ltv_simulation.mean(axis=0)
        ltv_std = ltv_simulation.std(axis=0)

        min_values = np.full(len(X_predict), Y_l[-1])
        max_values = ltv_mean + ltv_std

        complete_min_values = np.array(Y_l.tolist() + min_values.tolist())
        complete_max_values = np.array(Y_l.tolist() + max_values.tolist())

        new_df = pd.DataFrame()
        new_df["min_arpi"] = complete_min_values
        new_df["mean_arpi"] = (complete_max_values + complete_min_values) / 2
        new_df["max_arpi"] = complete_max_values
        new_df["age"] = X_complete
        new_df["platform"] = combination[0]
        new_df["country_group"] = combination[1]
        new_df["channel"] = combination[2]
        new_df["install_month"] = combination[3]
        new_df["eligable_prediction"] = eligable_prediction

        df_true_preds = pd.concat([df_true_preds, new_df])

        # plt.scatter(X_l, Y_l, color="r")
        # plt.fill_between(
        #     X_predict,
        #     min_values,
        #     max_values,
        #     color="b",
        #     alpha=0.2,
        # )
        # plt.show()

In [19]:
data_predict_g = data_predict[
    ["platform", "country_group", "channel", "install_month", "cpi"]
]
# Assuming you want to group by 'platform', 'country_group', 'channel', and 'install_month'
grouped_data = (
    data_predict_g.groupby(["platform", "country_group", "channel", "install_month"])[
        "cpi"
    ]
    .mean()
    .reset_index()
)

result = pd.merge(
    grouped_data[["cpi", "platform", "country_group", "channel", "install_month"]],
    df_true_preds,
    on=["platform", "country_group", "channel", "install_month"],
    how="right",
)


result = pd.merge(
    data_predict[
        ["platform", "country_group", "channel", "install_month", "spend", "installs"]
    ]
    .groupby(["platform", "country_group", "channel", "install_month"])[
        ["spend", "installs"]
    ]
    .sum()
    .reset_index(),
    result,
    on=["platform", "country_group", "channel", "install_month"],
    how="right",
)

In [20]:
result_filtered = result[
    (result.min_arpi / result.cpi > 1)
    | (result.max_arpi / result.cpi > 1)
    | (result.mean_arpi / result.cpi > 1)
]

result_filtered.sort_values(
    inplace=True, by=["platform", "country_group", "channel", "install_month", "age"]
)

<ipython-input-20-0a174ab836a3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_filtered.sort_values(


In [21]:
result_filtered_min = result_filtered[
    (result_filtered.min_arpi / result_filtered.cpi)
    > 1 | (result_filtered.eligable_prediction == True)
]
result_filtered_min["ltv/cac"] = (
    result_filtered_min["min_arpi"] / result_filtered_min["cpi"]
)

result_filtered_min.sort_values(
    by=["platform", "country_group", "channel", "install_month", "age"], inplace=True
)

result_filtered_min.rename(
    columns={
        "arpi": "Revenue per install",
        "spend": "Spend",
        "cpi": "Cost per install",
        "installs": "Number of installs",
        "age": "Cohort Age, Days",
        "min_arpi": "Min possible Avg. Revenue per install",
        "mean_arpi": "Mean possible Avg. Revenue per install",
        "max_arpi": "Max possible Avg. Revenue per install",
    },
    inplace=True,
)

result_filtered_min.drop(columns=["eligable_prediction"], inplace=True)

<ipython-input-21-801364fca751>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_filtered_min["ltv/cac"] = (
<ipython-input-21-801364fca751>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_filtered_min.sort_values(
<ipython-input-21-801364fca751>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_filtered_min.rename(
<ipython-input-21-801364fca751>:29: Setti